In [1]:
import httpx

# Comparing prod and dev

Ok, not much difference here.

In [2]:
prod = "https://wersdoerfer.de"
staging = "https://homepage.staging.django-cast.com"
dev = "http://localhost:8000"
blog_slug = "ephes_blog"
# blog_slug = "das_claas_und_nora_blog"

In [3]:
%%time
r = httpx.get(f"{prod}/blogs/{blog_slug}/")
r.raise_for_status()
content = r.text
print(len(content))

59186
CPU times: user 18.8 ms, sys: 4.46 ms, total: 23.3 ms
Wall time: 206 ms


In [4]:
%%time
r = httpx.get(f"{staging}/blogs/{blog_slug}/")
r.raise_for_status()
content = r.text
print(len(content))

81601
CPU times: user 21.6 ms, sys: 5.03 ms, total: 26.7 ms
Wall time: 191 ms


In [5]:
%%time
r = httpx.get(f"{dev}/blogs/{blog_slug}/")
r.raise_for_status()
content = r.text
print(len(content))

68942
CPU times: user 14.3 ms, sys: 2.92 ms, total: 17.2 ms
Wall time: 390 ms


In [6]:
%%time
r = httpx.get(f"{dev}/blogs/{blog_slug}/")
r.raise_for_status()
content = r.text
print(len(content))

68942
CPU times: user 13.8 ms, sys: 2.88 ms, total: 16.7 ms
Wall time: 460 ms


# How long does the serve call take?

In [9]:
from django.test.client import RequestFactory
from django_htmx.middleware import HtmxDetails

In [10]:
request_factory = RequestFactory()
request = request_factory.get("/")
request.htmx = HtmxDetails(request)

In [11]:
blog = Blog.objects.get(slug=blog_slug)

In [12]:
%%time
response = blog.serve(request).render()
print(len(response.content))

59238
CPU times: user 265 ms, sys: 58.4 ms, total: 324 ms
Wall time: 389 ms


/Users/jochen/.virtualenvs/homepage/lib/python3.12/site-packages/wagtail/blocks/base.py:233: RemovedInWagtail60Warning: GalleryBlock.get_template should accept a 'value' argument as first argument
  warnings.warn(


In [10]:
[b.slug for b in Blog.objects.all()]

['das_claas_und_nora_blog', 'ephes_blog']

In [11]:
#Rendition.objects.all().delete()

In [45]:
slug = "weeknotes-2023-07-24"
post = Post.objects.get(slug=slug)

In [46]:
post.sync_media_ids()

In [47]:
post.media_ids_from_db

{'image': set(), 'video': set(), 'gallery': {228}, 'audio': set()}

In [48]:
post.media_ids_from_body

{'gallery': {228}}

In [50]:
[gallery] = post.galleries.all()

In [51]:
gallery


<Gallery: Gallery object (228)>

In [52]:
gallery.images.all()

<ImageQuerySet [<Image: Marktplatz in Vannes>, <Image: Fachwerkhäuser in Vannes>, <Image: Fensterspiegelung>, <Image: Sonnenstern an der Saline>, <Image: Fernglas zum Beobachten der Vögel an der Saline>, <Image: Salz in der Saline>, <Image: Windrad neben der Saline>, <Image: Ein paar Blumen am Wegesrand>, <Image: Atlantikküste vor Quiberon>, <Image: Steilküste vor Quiberon>, <Image: Strand in Quiberon>, <Image: Am Strand von Mousterian>, <Image: Frau am Strand von Mousterian>]>

In [53]:
post.images.all()

<ImageQuerySet []>

In [55]:
Image.objects.count()

1734

## Count Queries

In [13]:
import sqlparse

from django.db import connection, reset_queries

In [14]:
def count_queries(func):
    reset_queries()
    response = func()
    print(len(response.content))
    return len(connection.queries)


def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)    

In [15]:
request_factory = RequestFactory()
request = request_factory.get("/")
request.htmx = HtmxDetails(request)

In [16]:
blog = Blog.objects.get(slug=blog_slug)

In [17]:
%%time
num_queries = count_queries(blog.serve(request).render)
print(num_queries)

5
CPU times: user 174 ms, sys: 22.5 ms, total: 196 ms
Wall time: 209 ms


In [18]:
show_queries(connection.queries)

SELECT "wagtailcore_site"."id",
       "wagtailcore_site"."hostname",
       "wagtailcore_site"."port",
       "wagtailcore_site"."site_name",
       "wagtailcore_site"."root_page_id",
       "wagtailcore_site"."is_default_site",
       CASE
           WHEN ("wagtailcore_site"."hostname" = 'testserver'
                 AND "wagtailcore_site"."port" = 80) THEN 0
           WHEN ("wagtailcore_site"."hostname" = 'testserver'
                 AND "wagtailcore_site"."is_default_site") THEN 1
           WHEN "wagtailcore_site"."is_default_site" THEN 2
           ELSE 3
       END AS "match",
       "wagtailcore_page"."id",
       "wagtailcore_page"."path",
       "wagtailcore_page"."depth",
       "wagtailcore_page"."numchild",
       "wagtailcore_page"."translation_key",
       "wagtailcore_page"."locale_id",
       "wagtailcore_page"."latest_revision_id",
       "wagtailcore_page"."live",
       "wagtailcore_page"."has_unpublished_changes",
       "wagtailcore_page"."first_published_at",
 